### Imports

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, classification_report
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Get data

In [ ]:
train_df = pd.read_csv('C:/Users/Samir/Documents/GitHub/Projet-STT-3795/src/data/wav_files_clean/train_preprocessed_data')
test_df = pd.read_csv('C:/Users/Samir/Documents/GitHub/Projet-STT-3795/src/data/wav_files_clean/test_preprocessed_data')
validation_df = pd.read_csv('C:/Users/Samir/Documents/GitHub/Projet-STT-3795/src/data/wav_files_clean/validation_preprocessed_data')

df = pd.concat([train_df, test_df, validation_df]).reset_index().iloc[:, 2:]
df.rename(columns={'0': 'label'}, inplace=True)

df_without_label = df.iloc[:, 0:-1]

df_without_label

### Principal components

In [3]:
def get_PCs(dataframe, percentage_variance):
    print()
    scaler = StandardScaler()
    scaled_df = scaler.fit_transform(dataframe)
    print(f'Scaled_df Mean = {np.mean(scaled_df)},\nScaled_df Std = {np.std(scaled_df)}')


    pca_T = PCA()
    pca_T.fit_transform(scaled_df)
    ev = pca_T.explained_variance_
    print()
    print(f'Total variance = {sum(ev)}')

    pca = PCA(percentage_variance/100)
    principal_components = pca.fit_transform(scaled_df)
    explained_variance = pca.explained_variance_
    percentage = sum(pca.explained_variance_ratio_)
    print(f'Real percentage = {percentage}')
    print(f'Variance for {round(percentage*100, 2)}% = {sum(explained_variance)}')
    print(f'Number of PCs for {round(percentage*100, 2)}% = {len(explained_variance)}')
    print(f'Attribute lost = {len(scaled_df[0]) - len(explained_variance)}')
    names = pca.get_feature_names_out()
    return pd.DataFrame(data=principal_components, columns=names)


In [ ]:
df_pca = get_PCs(df_without_label, 99)
df_pca['label'] = df['label']


## Models

### Training

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df_pca.iloc[:, 0:-1], df['label'], stratify=df['label'], test_size=0.1, random_state=42) 

# Model initialization
svm = SVC(verbose=3)
random_forest = RandomForestClassifier(verbose=3)
nn = MLPClassifier(verbose=3)

# Models fiting
print("Training SVM")
svm.fit(X_train, Y_train)
print("Training RFC")
random_forest.fit(X_train, Y_train)


### Predictions

In [15]:
# Models prediction
svm_predictions = svm.predict(X_test)
random_forest_predictions = svm.predict(X_test)

### Metrics

#### Accuracy

In [ ]:
svm_accuracy = accuracy_score(Y_test, svm_predictions)
random_forest_accuracy = accuracy_score(Y_test, random_forest_predictions)

print(f'SVM accuracy: {svm_accuracy}')
print(f'RDF accuracy: {random_forest_accuracy}')

#### Confusion matrix

In [ ]:
svm_conf_matrix = confusion_matrix(Y_test, svm_predictions)
rdf_conf_matrix = confusion_matrix(Y_test, random_forest_predictions)

#svm_cf_display = ConfusionMatrixDisplay(confusion_matrix=svm_conf_matrix, display_labels=[False, True])
#rdf_cf_display = ConfusionMatrixDisplay(confusion_matrix=rdf_conf_matrix, display_labels=[False, True])

print(svm_conf_matrix)
print()
print(rdf_conf_matrix)

#### Classification report

In [ ]:
target = df['label']
report = classification_report(Y_test, svm_predictions)
print(report)